In [1]:
import prep
import prsa
import peaks_detection
import os
import numpy as np
import biosppy

In [2]:
WINDOWSIZE = 3
FOLDER_NAME = f'results_L{WINDOWSIZE}' # folder to save results in .csv format

In [3]:
NORMO_DIR = './data_2/normokapnia'
HYPER_DIR = './data_2/hiperkapnia'

ABP_COLUMN = 'abp_finger_mm_hg_[abp_finger_mm_Hg_]'
ALTERNATIVE_ABP_COLUMN = 'abp_finger[abp_finger]'

In [4]:
normo_dc_data = {}
normo_ac_data = {}
hyper_dc_data = {}
hyper_ac_data = {}

normo_dc_w = {}
normo_ac_w = {}
hyper_dc_w = {}
hyper_ac_w = {}

In [5]:
for filename in os.listdir(NORMO_DIR):
    if os.path.isfile(os.path.join(NORMO_DIR, filename)):
        
        # read data
        file_path = os.path.join(NORMO_DIR, filename)
        try:
            df = prep.read_data(file_path, ABP_COLUMN, sep=';')
        except KeyError:
            df = prep.read_data(file_path, ALTERNATIVE_ABP_COLUMN, sep=';')

        # data preprocessing
        df.interpolate(method='linear', inplace=True)
        try:  # few files have to much noise to be filtered
            filtered_abp = biosppy.signals.abp.abp(signal=df['Values'], sampling_rate=200, show=False)[1]
            df['Values'] = filtered_abp
        except ValueError:
            print(f'Error in {filename}')
            continue

        # finding peaks
        rr = peaks_detection.rr_intervals(df['Values'])

        # calculating PRSA
        prsa_dc, dc_w = prsa.calculate(rr, WINDOWSIZE, "DC", 0.2)
        prsa_ac, ac_w = prsa.calculate(rr, WINDOWSIZE, "AC", 0.2)

        # calculating capacity
        dc= prsa.capacity(prsa_dc)
        ac = prsa.capacity(prsa_ac)

        # saving results
        filename = filename[-7:-5]
        normo_dc_data[filename] = dc
        normo_ac_data[filename] = ac
        normo_dc_w[filename] = dc_w
        normo_ac_w[filename] = ac_w
        print(f'Finished {filename}')

Finished 01
Finished 03
Finished 04
Finished 05
Finished 07
Finished 08
Finished 09
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28


c:\Users\Ignacy\PycharmProjects\Brainlab\venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Ignacy\PycharmProjects\Brainlab\venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Error in e[3]_[n]_[29].csv
Finished 30
Finished 31
Finished 32
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Finished 39
Finished 40
Finished 41
Finished 42
Finished 43
Finished 44
Finished 45
Finished 46
Finished 47
Finished 48
Finished 49
Finished 50
Finished 51
Finished 52
Finished 53
Finished 54
Finished 55
Finished 56


In [6]:
for filename in os.listdir(HYPER_DIR):
    if os.path.isfile(os.path.join(HYPER_DIR, filename)):
        
        # read data
        file_path = os.path.join(HYPER_DIR, filename)
        try:
            df = prep.read_data(file_path, ABP_COLUMN, sep=';')
        except KeyError:
            df = prep.read_data(file_path, ALTERNATIVE_ABP_COLUMN, sep=';')

        # data preprocessing
        df.interpolate(method='linear', inplace=True)
        try:  # few files have to much noise to be filtered
            filtered_abp = biosppy.signals.abp.abp(signal=df['Values'], sampling_rate=200, show=False)[1]
            df['Values'] = filtered_abp
        except ValueError:
            print(f'Error in {filename}')
            continue

        # finding peaks
        rr = peaks_detection.rr_intervals(df['Values'])

        # calculating PRSA
        prsa_dc, dc_w = prsa.calculate(rr, WINDOWSIZE, "DC", 0.2)
        prsa_ac, ac_w = prsa.calculate(rr, WINDOWSIZE, "AC", 0.2)

        # calculating capacity
        dc= prsa.capacity(prsa_dc)
        ac = prsa.capacity(prsa_ac)

        # saving results
        filename = filename[-7:-5]
        hyper_dc_data[filename] = dc
        hyper_ac_data[filename] = ac
        hyper_dc_w[filename] = dc_w
        hyper_ac_w[filename] = ac_w
        print(f'Finished {filename}')

Finished 01
Finished 03
Finished 04
Finished 05
Finished 07
Finished 08
Finished 09
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
Finished 30
Finished 31
Finished 32
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Finished 39
Finished 40
Finished 41
Finished 42
Finished 43
Finished 44
Finished 45
Finished 46
Finished 47
Finished 48
Finished 49
Finished 50
Finished 51
Finished 52
Finished 53
Finished 54
Finished 55
Finished 56


In [ ]:
def save_to_csv(file_tittle: str, folder_name: str, **columns: dict[str: float]):
    """
    Saves results to csv file
    :param file_tittle: name of the file
    :param folder_name: name of the folder
    :param columns: column names and their values for given file
    """

    # Get the column names and values
    columns_names = list(columns.keys())
    columns_values = list(columns.values())

    # Get the filenames
    filenames = set()
    for col in columns_values:
        filenames.update(list(col.keys()))

    filenames = list(filenames)
    filenames.sort(key=lambda x: int(x))
    # Create the directory if it doesn't exist
    os.makedirs(folder_name, exist_ok=True)

    with open(f'{folder_name}/{file_tittle}.csv', 'w') as f:
        f.write(f'file number;{";".join(columns_names)}\n')
        for filename in filenames:
            row_values = [row.get(filename, -np.inf) for row in columns_values]
            if -np.inf in row_values:
                continue
            f.write(f'{filename};{";".join([str(val) for val in row_values])}\n')

In [8]:
save_to_csv(
    file_tittle = 'AC_DC_Values',
    folder_name = FOLDER_NAME,
    AC_normo = normo_ac_data, 
    AC_hyper = hyper_ac_data,
    DC_normo = normo_dc_data, 
    DC_hyper = hyper_dc_data,
    )

In [9]:
save_to_csv(
    file_tittle = 'AC_DC_Windows',
    folder_name = FOLDER_NAME,
    AC_normo_W = normo_ac_w,
    AC_hyper_W = hyper_ac_w,
    DC_normo_W = normo_dc_w,
    DC_hyper_W = hyper_dc_w
    )